In [1]:
import pandas as pd
from openai import OpenAI
from autoddg import AutoDDG
from autoddg.utils import get_sample
from autoddg.evaluation import BaseEvaluator
from typing import Optional
# --- Import custom files ---
from prompts import ALL_RELATED_WORK_PROMPTS
from utils import log_result 

In [2]:
# --- LLM Config ---
MODEL_CONFIG = {
    "base_url": "http://localhost:11434/v1",
    "api_key": "ollama",
    "model_name": "llama3",
}

# --- Experiment Config ---
DATASET_NAME = "CODE-15%: a large scale annotated dataset of 12-lead ECGs"
DATA_FILE = "../src/autoddg/related/data/code-15.csv"
PAPER_FILE = "../src/autoddg/related/papers/code15.pdf"
RESULTS_FILE = "autoddg_experiment_results.csv"

# --- Define Evaluation Class ---
class Eval(BaseEvaluator):
    def __init__(self, model_name: str = MODEL_CONFIG["model_name"]):
        client = OpenAI(
            api_key=MODEL_CONFIG["api_key"], 
            base_url=MODEL_CONFIG["base_url"]
        )
        super().__init__(client=client, model_name=model_name)

# Initialize Core Tools
client = OpenAI(api_key=MODEL_CONFIG["api_key"], base_url=MODEL_CONFIG["base_url"])
auto_ddg = AutoDDG(client=client, model_name=MODEL_CONFIG["model_name"])
auto_ddg.set_evaluator(Eval())

In [ ]:
# Cell 3: Data Loading and Core Profiling (Run Once)

print("--- Loading Data and Running Core Profiling ---")
df = pd.read_csv(DATA_FILE)
sample_df, dataset_sample = get_sample(df, sample_size=100)

basic_profile, structural_profile = auto_ddg.profile_dataframe(df)
semantic_profile = auto_ddg.analyze_semantics(sample_df)
data_topic = auto_ddg.generate_topic(DATASET_NAME, None, dataset_sample)

print("Profiling Complete.")

--- Loading Data and Running Core Profiling ---


/opt/homebrew/Caskroom/miniconda/base/envs/nlp_final/lib/python3.12/site-packages/datamart_profiler/core.py:199: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.astype(object).fillna('').astype(str)


In [ ]:
# Cell 4: Define Prompts to Test (Selects from the imported dictionary)

# Define which prompts you want to run for this experiment
PROMPTS_TO_TEST = {
    "V1_Revised": ALL_RELATED_WORK_PROMPTS["V1_Revised"],
    # "V2_Aggressive": ALL_RELATED_WORK_PROMPTS["V2_Aggressive"],
    "V3_Hybrid": ALL_RELATED_WORK_PROMPTS["V3_Hybrid"]
}
print(f"Testing {len(PROMPTS_TO_TEST)} related work prompts.")



In [ ]:
# Cell 5: Run and Log Baseline (Vanilla) Description

print("\n--- Running Baseline (Vanilla) Test ---")
prompt_baseline, description_baseline = auto_ddg.describe_dataset(
    dataset_sample=dataset_sample,
    dataset_profile=basic_profile,
    use_profile=True,
    semantic_profile=semantic_profile,
    use_semantic_profile=True,
    data_topic=data_topic,
    use_topic=True,
    use_related_profile=False  # Vanilla
)

baseline_scores = auto_ddg.evaluate_description(description_baseline)
print(f"Baseline Scores: {baseline_scores}")

# Log result using the new utility function (passing required file/dataset args)
log_result(
    prompt_name="N/A", 
    description_type="Vanilla_AutoDDG", 
    description=description_baseline, 
    raw_scores=baseline_scores,
    dataset_name=DATASET_NAME,
    file_path=RESULTS_FILE
    # related_profile is omitted (defaults to None)
)

In [ ]:
# Cell 6: Run and Log Augmented Descriptions for Multiple Prompts

for prompt_name, extraction_prompt in PROMPTS_TO_TEST.items():
    print(f"\n--- Running Augmented Test with Prompt: {prompt_name} ---")
    
    # Step A: Analyze related work using the current prompt
    related_profile = auto_ddg.analyze_related(
        pdf_path=PAPER_FILE,
        dataset_name=DATASET_NAME,
        extraction_prompt=extraction_prompt,
        max_pages=10
    )
    print(f"Related Work Summary: {related_profile['summary'][:150]}...")

    # Step B: Generate description with the new related profile
    prompt_augmented, description_augmented = auto_ddg.describe_dataset(
        dataset_sample=dataset_sample,
        dataset_profile=basic_profile,
        use_profile=True,
        semantic_profile=semantic_profile,
        use_semantic_profile=True,
        data_topic=data_topic,
        use_topic=True,
        related_profile=related_profile,
        use_related_profile=True # Augmented
    )
    
    # Step C: Evaluate and Log
    augmented_scores = auto_ddg.evaluate_description(description_augmented)
    print(f"Augmented Scores ({prompt_name}): {augmented_scores}")
    
    log_result(
        prompt_name=prompt_name, 
        description_type="Augmented_AutoDDG", 
        description=description_augmented, 
        raw_scores=augmented_scores,
        dataset_name=DATASET_NAME,
        file_path=RESULTS_FILE,
        related_profile=related_profile # Now logs the profile!
    )
    
print("\nAll experiments complete. Results saved to:", RESULTS_FILE)